#### Lab 02 - Collaborative Filtering (User, Rating, Film)

In [ ]:
import pandas, warnings

warnings.filterwarnings("ignore")

In [ ]:
table_film = pandas.read_table("movies.csv", sep=",")

table_film.iloc[:5]

In [ ]:
rating = pandas.read_table("ratings.csv", sep=",")

rating.iloc[:5]